Find introns that are longer than the median coding gene length (1956nt according to WormBook)

In [41]:
with open('long_introns.bed','w') as outfile:   
    with open('collapse_new_10p_filtered_named.isoforms.bed') as bed:
        for line in bed:
            line=line.strip().split()
            starts=line[11][:line[11].rfind(',')].split(',')
            sizes=line[10][:line[10].rfind(',')].split(',')
            ends=[]
            for idx,start in enumerate(starts):
                end=int(start)+int(sizes[idx])
                ends.append(end)
            for idx,end in enumerate(ends):
                if len(ends)==idx+1:
                    continue
                else:
                    intron_length=int(starts[idx+1])-end
                    if intron_length>=1956:
                        outfile.write('\t'.join(line)+'\n')
                        break

Just make a list of intron lengths to see if there are any obvious fusions from that

In [7]:
with open('intron_lengths.tsv','w') as outfile:   
    with open('collapse_new_10p_filtered_named.isoforms.bed') as bed:
        for line in bed:
            line=line.strip().split()
            starts=line[11][:line[11].rfind(',')].split(',')
            sizes=line[10][:line[10].rfind(',')].split(',')
            ends=[]
            for idx,start in enumerate(starts):
                end=int(start)+int(sizes[idx])
                ends.append(end)
            for idx,end in enumerate(ends):
                if len(ends)==idx+1:
                    continue
                else:
                    intron_length=int(starts[idx+1])-end
                    outfile.write(line[3]+'\t'+str(intron_length)+'\n')

In [8]:
!sort -nrk 2,2 intron_lengths.tsv > intron_lengths.sorted.tsv

In [9]:
!head intron_lengths.sorted.tsv

Y71H2AM.5.2_WBGene00022170	86515
productive_splice_isoform_220_WBGene00011884	32121
ZK909.2h.1-0_WBGene00002189	21230
C37F5.1a.1_WBGene00002990	20948
Y41C4A.4c.1_WBGene00000793	18615
productive_splice_isoform_76_WBGene00000793	18615
T12A2.1.1_WBGene00020436	17240
Y71D11A.3a.1_WBGene00022104	17216
T28B11.1a.2_WBGene00012117	17091
ZK1073.2.1_WBGene00014214	16515


In [13]:
def find_wb_id(x):
	start = x.index('gene_id') + len('gene_id')+2
	start_to_end = x[start:]
	end = start_to_end.index(';')-1
	return start_to_end[:end]

def find_biotype(x):
	start = x.index('gene_biotype') + len('gene_biotype')+2
	start_to_end = x[start:]
	end = start_to_end.index(';')-1
	return start_to_end[:end]

with open('exons.bed','w') as outfile:
    with open('/expanse/lustre/scratch/eschiksn/temp_project/IAN_MAPPING/genome_files/c_elegans.PRJNA13758.WS279.canonical_geneset.sorted.gtf') as gtf:
        for line in gtf:
            line=line.strip().split('\t')
            if line[2]=='exon' and find_biotype(line[8])=='protein_coding':
                wb=find_wb_id(line[8])
                chrom,start,end,strand=line[0],str(int(line[3])+1),line[4],line[6]
                out=[chrom,start,end,wb,'60',strand]
                if int(start)<int(end):
                    outfile.write('\t'.join(out)+'\n')

In [15]:
!bedtools intersect -a long_introns.bed -b exons.bed -split -wb -wa -s > tmp.bed

In [10]:
!bedtools intersect -a long_introns.bed -b exons.bed -wb -wa -s > tmp_unsplit.bed

In [11]:
gene_overlap_dict={}

with open('tmp.bed') as intersect:
    for line in intersect:
        line=line.strip().split()
        isoform=line[3]
        overlap=line[15]
        if isoform not in gene_overlap_dict:
            gene_overlap_dict[isoform]=[line[15]]
        elif line[15] not in gene_overlap_dict[isoform]:
                gene_overlap_dict[isoform].append(line[15])

In [12]:
gene_overlap_dict={}

with open('tmp_unsplit.bed') as intersect:
    for line in intersect:
        line=line.strip().split()
        isoform=line[3]
        overlap=line[15]
        if isoform not in gene_overlap_dict:
            gene_overlap_dict[isoform]=[line[15]]
        elif line[15] not in gene_overlap_dict[isoform]:
                gene_overlap_dict[isoform].append(line[15])

In [13]:
with open('long_intron_multi_gene_isoforms_unsplit.txt','w') as outfile:
    for isoform in gene_overlap_dict:
        if len(gene_overlap_dict[isoform])>=2:
            outfile.write(isoform+'\n')

In [20]:
with open('long_intron_multi_gene_isoforms.txt','w') as outfile:
    for isoform in gene_overlap_dict:
        if len(gene_overlap_dict[isoform])>=2:
            outfile.write(isoform+'\n')